# ロジスティクス回帰をラフに試す

## DataFrameの作成

作り方は scripts/netkeiba/create_race_result_data_frame.py 参照

In [ ]:
import pandas as pd
from pathlib import Path
from keiba_machine_learning.netkeiba.constants import RACE_DATA_DIR

pd.set_option('display.max_columns', 100)

# Data Frame の作成
df = pd.read_pickle(Path(RACE_DATA_DIR) / 'race_results_data_frame.pickle').copy()

#display(df)

## 着順データの整理

馬券に絡むか絡まないかだと4着以下は4着でも18着でも同じようなものなのでまとめてしまう

In [ ]:
df['order_of_placing'] = df['order_of_placing'].map(lambda n: n if n <= 3 else 4)
#df['order_of_placing'].value_counts()

## ダミー変数化

In [ ]:
# 使わない項目を削除
# 例えば hourse_id や horuse_name などはユニークな値 10,000 件以上あるのでこう言うのは今回は（素振り段階では）ダミー変数にしない
df.drop(['race_time', 'race_id', 'horse_id', 'horse_name', 'jockey_name', 'race_number', 'starts_at'], axis=1, inplace=True)

# ダミー変数化するものだけ引数で指定
dummy_df = pd.get_dummies(df, columns=['race_track', 'track_kind', 'track_direction', 'race_distance_by_meter', 'track_surface', 'weather', 'bracket_number', 'horse_number', 'horse_age', 'horse_gender', 'impost', 'jockey_id', 'win_betting_ratio', 'favorite_order', 'horse_weight', 'weight_change'])
#display(dummy_df)

## 訓練データとテストデータに分ける

In [ ]:
from sklearn.model_selection import train_test_split

x = dummy_df.drop(['order_of_placing'], axis=1)
y = dummy_df['order_of_placing'].astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.3, random_state=0)

## アンダーサンプリング

着順の整理で4位の割合が大幅に増えてしまったので、標本を調整しないと4位予想に偏ってしまう

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]
#変更点
rus = RandomUnderSampler(
    sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1}, random_state=71
)
x_train_rus, y_train_rus = rus.fit_resample(x_train.values, y_train.values)

## 訓練

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train_rus, y_train_rus)

## 制度の確認

素振りなので低精度でも気にしない

In [ ]:
print(model.score(x_train, y_train), model.score(x_test, y_test))

## 予測

In [ ]:
y_pred = model.predict(x_test)
#print(y_pred)

## 予測結果確認

In [ ]:
pred_df = pd.DataFrame({"pred": y_pred, "actual": y_test})
display(pred_df)  # 予想と実際の着順の対応表
pred_df[pred_df["pred"] == 1]["actual"].value_counts()  # 1着と予想した際に実際にどの着順だったか

## 回帰係数の確認

In [ ]:
coefs = pd.Series(model.coef_[0], index=x.columns).sort_values()
display(coefs)